# import packages


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import os

from tensorflow import keras
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Gether Data from kaggle

In [2]:
# ! mkdir ~/.kaggle
# ! cp /content/kaggle.json ~/.kaggle
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d tongpython/cat-and-dog

In [3]:
# ! unzip /content/cat-and-dog.zip

In [4]:
# ! rm /content/cat-and-dog.zip

# Data Preprocessing

In [5]:
path = './training_set/training_set'
testing = './test_set/test_set'

# image_generator = ImageDataGenerator(rescale=1.0/255)    

# train_dataset = image_generator.flow_from_directory(batch_size=32,
#                                                  directory=path,
#                                                  shuffle=True,
#                                                  target_size=(32, 32), 
#                                                  class_mode='binary')

# validation_dataset = image_generator.flow_from_directory(batch_size=32,
#                                                  directory=testing,
#                                                  shuffle=True,
#                                                  target_size=(32, 32), 
#                                                  class_mode='binary')

In [6]:
# len(train_dataset), len(validation_dataset)

In [7]:
image_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)   

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory=path,
                                                 shuffle=True,
                                                 target_size=(224, 224), 
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory=testing,
                                                 shuffle=True,
                                                 target_size=(224, 224), 
                                                 class_mode='categorical')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [8]:
img, label = next(train_dataset)

# Building the model

In [9]:
model = keras.Sequential()

In [10]:
# Adding Convoluation layers
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2),  strides=2, padding='valid'))
model.add(keras.layers.Dropout(0.2))

# 2nd
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(keras.layers.Dropout(0.2))

# 3rd
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Flatten())

In [11]:
# Dense layers

model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(units=2, activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 56, 56, 32)        0

In [13]:
# compiling the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(train_dataset, epochs=10, validation_data=validation_dataset)


Epoch 1/32
251/251 [==============================] - 40s 148ms/step - loss: 0.7099 - accuracy: 0.5790 - val_loss: 0.5974 - val_accuracy: 0.6486
Epoch 2/32
251/251 [==============================] - 33s 132ms/step - loss: 0.6105 - accuracy: 0.6491 - val_loss: 0.5799 - val_accuracy: 0.6788
Epoch 3/32
251/251 [==============================] - 33s 132ms/step - loss: 0.5768 - accuracy: 0.6827 - val_loss: 0.5423 - val_accuracy: 0.7100
Epoch 4/32
251/251 [==============================] - 33s 132ms/step - loss: 0.5339 - accuracy: 0.7327 - val_loss: 0.5410 - val_accuracy: 0.7331
Epoch 5/32
251/251 [==============================] - 33s 132ms/step - loss: 0.4861 - accuracy: 0.7653 - val_loss: 0.5022 - val_accuracy: 0.7536
Epoch 6/32
251/251 [==============================] - 33s 132ms/step - loss: 0.4322 - accuracy: 0.7961 - val_loss: 0.4973 - val_accuracy: 0.7664
Epoch 7/32
251/251 [==============================] - 34s 134ms/step - loss: 0.3845 - accuracy: 0.8257 - val_loss: 0.5122 - val_ac

# mobile net v2

In [19]:
mobile_new = tf.keras.applications.mobilenet.MobileNet()

In [20]:
x = mobile_new.layers[-6].output
output = keras.layers.Dense(units=2, activation='softmax')(x) 

In [21]:
Models = keras.Model(inputs=mobile_new.input, outputs=output)

In [22]:
for layer in Models.layers[:-23]:
    layer.trainable = False

In [24]:
Models.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])